# Functions for monte-carlo simulation 

In [1]:
import numpy as np
from numba import jit
from scipy import stats
import matplotlib.pyplot as plt 


@jit(nopython=True)
def rho(T1, T2):
        Tmin = np.min(np.array([T1, T2]))
        Tmax = np.max(np.array([T1, T2]))
        c1 = 1 - np.cos(np.pi/2-0.366 *
                np.log(Tmax/(np.max(np.array([Tmin, 0.109])))))
        if Tmax < 0.2:
            c2 = 1-0.105*(1-1/(1+np.exp(100*Tmax-5))) *\
                ((Tmax-Tmin)/(Tmax-0.0099))
        else:
            c2 = 0
        if Tmax < 0.109:
            c3 = c2
        else:
            c3 = c1
        c4 = c1 + 0.5*(np.sqrt(c3)-c3)*(1+np.cos(np.pi*Tmin/0.109))

        if Tmax < 0.109:
            return c2
        elif Tmin > 0.109:
            return c1
        elif Tmax < 0.2:
            return np.min(np.array([c2, c4]))
        else:
            return c4


@jit(nopython=True)
def covariance(T, stdlnsaT):
    cov = np.zeros((len(T), len(T)))
    for i in range(len(T)):
        for j in range(len(T)):
            if i == j:
                cov[i, j] = stdlnsaT[i]**2
            else:
                cov[i, j] = rho(T[i], T[j]) * stdlnsaT[i] * stdlnsaT[j]
    return cov

def simulation(T, medlnsaT, stdlnsaT):
    cov_mat = covariance(T, stdlnsaT)
    z = np.linalg.cholesky(cov_mat)
    normls = np.random.normal(size=1000)
    normal = [values for values in normls if values>-1 and values < 1]
    normal_final = np.random.choice(normal,len(T),False)
    
    #sim = medlnsaT + np.matmul(z, np.random.normal(size=len(medlnsaT)))
    sim = medlnsaT + np.matmul(z,normal_final)
    return sim

def suite_response_spectra(T, medlnsaT, stdlnsaT, n):
    suite = np.zeros((n,len(T)))
    for i in range(n):
        suite[i,:] = simulation(T, medlnsaT, stdlnsaT)
        while sum(suite[i,:] < 0) >= 1:
            suite[i,:] = simulation(T, medlnsaT, stdlnsaT)
    return suite

def selection_criteria(T, target_med, target_std, data):
    weights = np.array([1, 2])
    devMeanSim = np.mean(np.log(data), axis=0)-np.log(target_med)
    devSkewSim = stats.skew(np.log(data), axis=0)
    devSigSim = np.std(np.log(data), axis=0) - \
                    np.sqrt(np.diagonal(covariance(T, target_std)))
    devTotalSim = weights[0]*np.sum(devMeanSim**2) + \
                    weights[1]*np.sum(devSigSim**2)+0.1 * \
                    (weights[0]+weights[1])*np.sum(devSkewSim**2)
    return np.min(np.abs(devTotalSim))


def best_suite(T, medlnsaT, stdlnsaT, n=20, ntrials=5):
        first_suite = suite_response_spectra(T, medlnsaT, stdlnsaT, n)
        criteria = selection_criteria(T, medlnsaT, stdlnsaT, first_suite)
        bestsuite = first_suite.copy()
        for suite in range(ntrials):
            next_suite = suite_response_spectra(T,medlnsaT, stdlnsaT, n)
            next_criteria = selection_criteria(T, medlnsaT, stdlnsaT, next_suite)            
            if next_criteria < criteria:
                criteria = next_criteria
                bestsuite = next_suite.copy()
        return bestsuite
    

@jit(nopython=True)    
def SSE(target,sample):
    SSE_sum = 0
    for i in range(len(target)):
        SSE_sum += (np.log(target[i])-np.log(sample[i]))**2
    return SSE_sum

@jit(nopython=True)
def SSEs(target_mean, target_std, sample_mean, sample_std, w = 1.0):
    SSEs_sum = 0 
    for i in range(len(target_mean)):
        SSEs_sum +=  (np.log(target_mean[i])-np.log(sample_mean[i]))**2 \
                    + w * (np.log(target_std[i])-np.log(sample_std[i]))**2
    return SSEs_sum


@jit(nopython=True)
def select_best_sample(population,target):
    match = population[0]
    sse_old = SSE(target,match)
    n = 0
    for i in range(1,len(population)):
        match_new = population[i]
        sse_new = SSE(target,match_new)
        if sse_new < sse_old:
            sse_old = sse_new
            n = i
            match = match_new.copy()     
    return match, n

@jit(nopython=True)
def select_best_samples(population,targets):
    m,n = targets.shape[0],targets.shape[1]
    matches = np.zeros((m,n))
    indec = np.zeros(m, dtype=np.int32)
    for i in range(m):
        target = targets[i]
        match, key = select_best_sample(population,target)
        indec[i] = key
        matches[i] = match
    return matches, indec 


@jit(nopython=True)
def greedy_optimization(population, samples, target_mean, target_std):
    match, indec = select_best_samples(population, samples)
    sample_mean, sample_std = suite_statistics(match)
    SSE_old = SSEs(target_mean, target_std, sample_mean, sample_std)
    m,n = match.shape[0], population.shape[0]
    match_new = match.copy()
    indec_new = indec.copy()
    for i in range(m):
        for j in range(n):            
            match_new[i] = population[j]
            indec_new[i] = j
            sample_mean, sample_std = suite_statistics(match_new)
            SSE_new = SSEs(target_mean, target_std, sample_mean, sample_std)
            if SSE_new < SSE_old:
                match = match_new.copy()
                SSE_old = SSE_new
                indec = indec_new.copy()
    return match, indec        
    
            
@jit(nopython =True)
def mean(data):
    sum = 0 
    for i in range(len(data)):
        sum += data[i]
    return sum/len(data)

@jit(nopython=True)
def deviation(data):
    sum = 0
    for i in range(len(data)):
        sum += (data[i]-mean(data))**2
    return np.sqrt(sum/(len(data)))

@jit(nopython=True)
def suite_statistics(data):
    trans_data = data.transpose()
    m,n = trans_data.shape[0], trans_data.shape[1]
    mean_row = np.zeros(m)
    std_row = np.zeros(m)
    for i in range(m):
        data_row = np.zeros(n)
        for j in range(n):
            data_row[j] = trans_data[i,j]
        mean_row[i] = np.exp(np.mean(np.log(data_row)))
        std_row[i] = np.exp(np.std(np.log(data_row)))
    return mean_row, std_row


def target(T):
    if T <= 0.1:
        return 1+15*T
    elif T >= 0.1 and T <= 0.67:
        return 2.5
    elif T >= 0.67 and T <= 4.0:
        return 1.67/T
    

In [22]:
from scipy.io import loadmat
path = "C:\\Users\\swopnil\\Downloads\\rec_selection_meta_data\\rec_selection_meta_data"
data = loadmat(path)
data["Periods"][0][:90]
data['Sa_1']

array([[1.52395e-01, 1.56035e-01, 1.57727e-01, ..., 5.86000e-04,
        5.19000e-04, 4.66000e-04],
       [4.75000e-02, 4.86000e-02, 4.92000e-02, ..., 1.90000e-05,
        1.52000e-05, 1.43000e-05],
       [4.36000e-02, 4.39000e-02, 4.40000e-02, ..., 1.53000e-04,
        1.37000e-04, 1.24000e-04],
       ...,
       [1.80320e-01, 2.06300e-01, 2.36459e-01, ..., 3.70000e-03,
        3.30000e-03, 2.83000e-03],
       [5.66000e-02, 5.71000e-02, 5.73000e-02, ..., 5.97000e-04,
        5.09000e-04, 5.06000e-04],
       [4.24000e-02, 4.31000e-02, 4.34000e-02, ..., 4.06000e-04,
        3.60000e-04, 3.30000e-04]])

# Randomly Distributed Standard Deviations and Target Response Spectrum

In [5]:
from scipy.io import loadmat
path = "C:\\Users\\admin\\Downloads\\CS_Selection-master\\CS_Selection-master\\Databases\\NGA_W2_meta_data"
data = loadmat(path)

In [160]:
T = data["Periods"][0][:90]

path2 = "D:\\Works\\Working Papers\\UHS.txt"
data2 = np.genfromtxt(path2)
T2,sa2 = data2[:,0],data2[:,1]

#T2 = np.array([0.01,0.05,0.1,0.5,1.,1.5,2.,2.5,3.])
#sa2 = np.array([0.55958,0.70106,1.12320,1.27170,0.65845,0.40510,0.24732,0.18413,0.13622])
sa2_smooth = np.interp(T,T2,sa2)

sa_t = np.array([0.05*target(Ti) for Ti in T])
std = np.random.lognormal(size = 10000)
std_filtered = [stdi for stdi in std if stdi > 0.1 and stdi < 1]
indices = np.random.randint(0,len(std_filtered),len(T))

std_sample = np.zeros(len(T))
for i in range(len(indices)):
    std_sample[i] = std_filtered[indices[i]]

#std_final = np.sort(std_sample)[::-1]
std_final = np.sort(std_sample)
std_final = np.sort(np.random.uniform(0.1,0.4,97))[::-1]

#std_final = np.random.lognormal(size=len(T))
#plt.plot(T,sa_t)
plt.plot(T,sa2_smooth)
plt.show()


#std_final


# Best simulated response_spectra

In [94]:
plt.rcParams["figure.figsize"] = (6.5,4)
#sim = suite_response_spectra(T,sa_t,std_final,20)
#sim = best_suite(T,sa_t,std_final,30,5)
sim = best_suite(T,sa2_smooth,std_final,150,5)
for i in range(len(sim)):
    plt.plot(T,sim[i],"grey",linewidth=0.8)
#plt.plot(T,sa_t,"black",linewidth=2.0,label="Target response specturm (IS 1893 soft soil)")
plt.plot(T,sa2_smooth,"black",linewidth=2.0,label="Target response specturm (Ktm 475 yrs)")
plt.plot(T,sim[2], "grey", linewidth=0.8, label="Simulated response spectra")
plt.plot(T,np.exp(np.mean(np.log(sim),axis=0)),"r--",label = "Simulated mean",linewidth=1.5)
plt.legend()
plt.xlabel("Period (sec)",fontsize = 11)
plt.ylabel("pSa (g)", fontsize = 11)
plt.show()




# Writing to files

In [95]:
#len(data["Sa_RotD50"])
population = np.zeros((len(data["Sa_1"]),90))
for i in range(len(population)):
    population[i] = data["Sa_1"][i][:90]



In [97]:
matches,indices = select_best_samples(population,sim)
#matches, indices = greedy_optimization(population, sim, sa_t, std_final)

In [27]:
#import graphviz
#from dask import delayed
#greedy = delayed(greedy_optimization)
#matches, indices = greedy(population, sim, sa_t, std_final).compute()



In [98]:
import matplotlib
%matplotlib
from matplotlib.pyplot import rcParams 
label_size = 10
rcParams['figure.figsize'] = 6,4
rcParams['xtick.labelsize'] = label_size
rcParams['ytick.labelsize'] = label_size
rcParams['legend.fontsize'] = label_size
rcParams['axes.labelsize'] = label_size
rcParams['font.family'] = 'Liberation Serif'

ax = plt.subplot(111)
for values in matches:
    ax.plot(T,values,color = "b",linewidth=1)
ax.plot(T,sa2_smooth,"black",linewidth=1.5,color = "black", label="10% probablity of exceedance in 50 years")
#plt.plot(T,sa_t,"black",linewidth=2.0,label="Ktm 475 yrs")
ax.plot(T,matches[9],linewidth=1,color = "b", label = "NGA West database")
ax.plot(T,np.mean(matches,axis=0),linestyle='--',color = "gray",linewidth=2, label = "NGA West mean")
ax.set_xlabel("Period [secs]",fontweight="bold")
ax.set_ylabel("Sa [g]",fontweight="bold")
ax.legend()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.show()



Using matplotlib backend: Qt5Agg


# Filter out ground motions that are repeated 

In [103]:
indices
index = []
index.append(indices[0])
for values in indices:
    if values not in index:
        index.append(values)


962

In [111]:
index

[1595,
 726,
 1548,
 810,
 1506,
 406,
 766,
 1632,
 179,
 962,
 567,
 528,
 450,
 1084,
 557,
 1511,
 1076,
 951,
 539]

In [167]:
import pandas as pd
datas3 = np.zeros((len(index),len(T)))
for i in range(len(index)):
    datas3[i] = data["Sa_1"][index[i]][:90]
    
datas4 = np.transpose(datas3)  
df = pd.DataFrame(datas4, columns=[values for values in index])
df['mean'] = df.mean(axis=1)
df1 = pd.DataFrame(T, columns=['T'])
#df1 = df1.append(df)
df1 = df1.join(df)
df1 = df1.join(pd.DataFrame(sa2_smooth, columns=['UHS']))

plt.plot(df1.iloc[:,0],df1.iloc[:,1:], linewidth =0.5, color='g')
plt.plot(df1.iloc[:,0],df1['mean'],linewidth=4,ls='--')
plt.plot(df1.iloc[:,0],df1['UHS'], linewidth=4,ls='-.')
#df1.to_clipboard(sep=',', index=False)


In [110]:
np.transpose(datas3)


array([[0.969834, 1.02896 , 1.05483 , ..., 0.0866  , 0.0828  , 0.0806  ],
       [0.693391, 0.689299, 0.698131, ..., 0.0336  , 0.0302  , 0.0258  ],
       [1.01414 , 1.07089 , 1.09542 , ..., 0.0959  , 0.103801, 0.101418],
       ...,
       [0.899902, 0.906361, 0.949129, ..., 0.0765  , 0.0644  , 0.0562  ],
       [0.628882, 0.618089, 0.627922, ..., 0.0394  , 0.0363  , 0.033   ],
       [0.493067, 0.50775 , 0.499109, ..., 0.026   , 0.0235  , 0.0213  ]])

In [41]:
T2 = np.array([0.01,0.1,0.2,0.3,0.5,1.0,2.0,3.,4.])
sa2 = np.array([0.547,1.672,1.698,1.446,0.903,0.438,0.18,0.103,0.06])
sa2_smooth = np.interp(T,T2,sa2)
plt.plot(T2,sa2,T,sa2_smooth,'--')
plt.show()

In [89]:
indices

array([ 726, 1595, 1506, 1076, 1595,  179, 1079, 1595, 1511, 1548, 1595,
        406,  406, 1632, 1548, 1548, 1632, 1595,  726,  406,  951, 1632,
       1548, 1595,  406, 1632,  450, 1548, 1595, 1632,  766,  726, 1595,
       1595,  406, 1595,  726,  406, 1595, 1595, 1632, 1548, 1548, 1595,
        726,  528, 1595, 1548, 1595, 1595,  726,  528, 1506, 1548, 1632,
       1595,  751, 1595, 1548, 1595, 1548, 1548, 1595, 1632,  810,  179,
       1548, 1548, 1511,  406,  567, 1595, 1595, 1595, 1632,  726,  539,
       1595, 1595,  406,  567,  450,  406,  179, 1084, 1511, 1076,  726,
        528, 1595, 1595, 1595, 1548, 1595, 1548,  179, 1548, 1511,  726,
       1548])